# Onderzoeksvragen en eerste verkenning: FIFA 19

<b> Redouan el Hidraoui <br>
Maurits Arissen <br>
Eric Houdijk <br>
V2A </b>

Lever één Jupyter Notebook in met de volgende onderdelen:
* een eerste verkenning en analyse van je dataset (stappen 1 tot en met 4 uit het data science proces);
* drie concept-onderzoeksvragen;
* een externe dataset (als bijlage in csv-formaat).

## Data collection

*Data collection is al voor je gedaan, aangezien je een dataset krijgt toegewezen, 
waarbij de ruwe data al eerder is verzameld door derden.
Je moet nog wel op zoek naar een externe dataset om je data mee uit te breiden. 
Deze data is echter ook al eerder voor je verzameld: je hoeft zelf geen dataverzameling te doen door bijvoorbeeld interviews te houden of sensoren uit te lezen.*
Na deze stap is er sprake van <b>ruwe data</b>.

In [ ]:
# imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import holoviews as hv

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics


hv.extension('bokeh')

Onze gekozen CSV-file dataset inladen: <br>
- fifa19.csv

De dataset bevat: <br>
1. Alle spelers uit FIFA 19.
2. Speelpositie en fysieke eigenschappen.
3. Persoonlijke spelersinformatie (bijv. nationaliteit, club, leeftijd, salaris etc.).

In [ ]:
fifa_source = pd.read_csv('source/fifa19.csv', index_col=0)

De grootte van onze dataset:
- 8.71 MB
- Bevat 18207 rijen (dus 18206 spelers, want de eerste rij bevat alle kolommen)

In [ ]:
rows_dataset = fifa_source.shape[0] - 1
rows_dataset

Een aantal rijen als voorbeeld voor hoe de dataset eruit ziet:

In [ ]:
fifa_source.head(10)

## Data processing

*Na het verzamelen van de data, moet je dit waarschijnlijk eerst bewerken, 
want het komt niet altijd in het formaat binnen dat je voor ogen had. 
Deze stap data processing is ook al grotendeels voor je gedaan: je krijgt de data immers in nette tabelvorm aangeleverd (bijvoorbeeld in CSV-files). 
Het is wel goed mogelijk dat je de data moet omvormen om meerdere datasets te kunnen combineren. 
Na deze stap heb je <b>geordende data</b>.*

De column <b>'Wage'</b>, de column <b>'Value'</b> en column <b>'Release Clause'</b> bevat characters zoals €, M en K. Om later gebruik te maken van deze data,
willen wij graag van deze characters af.

In [ ]:
fifa_source['Wage']

In [ ]:
fifa_source['Release Clause']

In [ ]:
fifa_source['Value']

Nu het verwijderen van de ongewenste characters <b>(Release Clause)</b>:

In [ ]:
fifa_source['Clean Release Clause'] = fifa_source['Release Clause']
fifa_source['Clean Release Clause'].replace(regex=True,inplace=True,to_replace=r'€',value=r'')

fifa_source['Clean Release Clause'] = fifa_source['Clean Release Clause'].replace(r'[KM]+$', '', regex=True).astype(float) * \
             fifa_source['Clean Release Clause'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int)

De <b>Release Clause</b> van de spelers na het opschonen:

In [ ]:
fifa_source['Clean Release Clause']

Nu het verwijderen van de ongewenste charachters <b>(Wage)</b>:

In [ ]:
fifa_source['Clean Wage'] = fifa_source['Wage']
fifa_source['Clean Wage'].replace(regex=True,inplace=True,to_replace=r'€',value=r'')

fifa_source['Clean Wage'] = fifa_source['Clean Wage'].replace(r'[KM]+$', '', regex=True).astype(float) * \
             fifa_source['Clean Wage'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int)

De <b>Wage</b> van de spelers na het opschonen:

In [ ]:
fifa_source['Clean Wage']

Nu het verwijderen van de ongewenste charachters <b>(Value)</b>:

In [ ]:
fifa_source['Clean Value'] = fifa_source['Value']
fifa_source['Clean Value'].replace(regex=True,inplace=True,to_replace=r'€',value=r'')

fifa_source['Clean Value'] = fifa_source['Clean Value'].replace(r'[KM]+$', '', regex=True).astype(float) * \
             fifa_source['Clean Value'].str.extract(r'[\d\.]+([KM]+)', expand=False).fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int)

De <b>Value</b> van de spelers na het opschonen:

In [ ]:
fifa_source['Clean Value']

## Data cleaning

*Je hebt nu een geordende dataset, maar deze is niet noodzakelijk al helemaal schoon. 
In de stap data cleaning ga je op zoek naar dubbele en missende waarden, outliers en andere onvolkomenheden en los je deze op. 
Na deze stap heb je <b>schone data</b>.*

Op dit moment bevat onze gekozen dataset <b>88 kolommen</b>:

In [ ]:
print('Aantal kolommen in fifa19.csv: {}'.format(len(fifa_source.columns)))
fifa_source.columns

Wij als team zijn tot de conclusie gekomen dat de volgende kolommen overbodig, en dus niet nodig zijn:
1. <b>Photo</b>, omdat: deze kolom een referentie bevat naar een afbeelding van de desbetreffende speler. Dit is nog relevant voor ons onderzoek nog hebben wij de beschikking tot deze foto's.
2. <b>Flag</b>, omdat: deze kolom een referentie bevat naar een afbeelding van de vlag die bij de nationaliteit hoort van de desbetreffende speler. Dit is nog relevant voor ons onderzoek, de 'Nationality' kolom is bruikbaarder, nog hebben wij beschikking tot deze afbeeldingen.
3. <b>Club logo</b>, omdat: bevat url's naar afbeeldingen
4. <b>Special</b>, omdat: nummers zonder duidelijke betekenis
5. <b>Body Type</b>, omdat: niet al te nuttig, erg generiek, bijna alle spelers delen dezelfde 3 bodytypes
6. <b>International Reputation</b>, omdat: weinig zeggend nummer
7. <b>Skill Moves</b>, omdat: weinig zeggend nummer
8. <b>Real Face</b>, omdat: Yes / No kolom, geen nuttige data
9. <b>Jersey Number</b>, omdat: weinig zeggend nummer, heeft geen specifiek betekenis

In [ ]:
fifa_source.drop(['Photo', 'Flag', 'Club Logo', 'Special', 'Body Type', 'International Reputation', 'Skill Moves', 'Real Face', 'Jersey Number'], axis=1, inplace=True)

De resterende kolommen na het verwijderen van de overbodige kolommen:

In [ ]:
print('Aantal kolommen in fifa19.csv na het verwijderen van overbodige kolommen: {}'.format(len(fifa_source.columns)))
fifa_source.columns

De column <b>'Overall'</b> bevat een getal (rating) van een speler. dit geeft weer hoe goed een speler
is t.o.v. de andere spelers binnen de dataset. Wij vinden dan ook dat deze column <b>niet</b> NaN mag zijn,
het heeft een belangrijke betekenis.
* Alle spelers die hier <b>NaN</b> hebben staan, worden uit onze dataset gehaald.

In [ ]:
aantal_nan_overall = pd.isnull(fifa_source['Overall']).sum().sum()
print('Aantal NaN-waardes column \'Overall\': {}'.format(aantal_nan_overall))

Onze dataset bevat geen spelers met NaN waardes in  column <b>'Overall'</b>.

De resterende columns met NaN worden door ons gevult met het getal 0. 
Dit doen wij om later (mochten die ontstaan door dit) foutmeldingen te voorkomen.

In [ ]:
aantal_nan = pd.isnull(fifa_source).sum().sum()
print('Huidige aantal NaN-waardes: {}'.format(aantal_nan))

In [ ]:
fifa_source.fillna(0, inplace=True)

In [ ]:
aantal_nan = pd.isnull(fifa_source).sum().sum()
print('Aantal NaN-waardes na het replacen met getal 0: {}'.format(aantal_nan))

Aantal duplicate spelers binnen onze fifa19.csv dataset:
* <b>0 spelers</b>

Dit checken wij aan de hand van het unieke ID die iedere speler toegewezen heeft gekregen.

In [ ]:
duplicatePlayers = fifa_source[fifa_source.duplicated(['ID'])]
print("Duplicate players gebaseerd op het unieke ID:", duplicatePlayers.count().sum())

## Data exploration & analysis

*Voordat je kan beginnen met het beantwoorden van de onderzoeksvragen, 
moet je de dataset eerst doorgronden en analyseren. 
Je moet begrijpen waar je data uit bestaat en hoe je met je data de onderzoeksvragen oplost. 
Gebruik <b>statistische analyses en visualisaties</b> om inzicht te krijgen in de dataset.* 

<b>Statische analyse column 'Overall'</b>:

In [ ]:
np.round(fifa_source['Overall'].describe(), decimals=2)

<b>Statische analyse column 'Age'</b>:

In [ ]:
np.round(fifa_source['Age'].describe(), decimals=0)

<b>Statische analyse column 'Release Clause'</b>:

In [ ]:
fifa_source['Release Clause'].describe()

<b>Histogram met Aantal spelers per 'Overall' waarde</b>:

In [ ]:
histogram = plt.hist(fifa_source['Overall'], edgecolor='black', linewidth=1.2)
plt.xlabel('Overall')
plt.ylabel('Amount')
histogram

<b>Histogram met Aantal spelers per 'Age' waarde</b>:

In [ ]:
histogram = plt.hist(fifa_source['Age'], edgecolor='black', linewidth=1.2)
plt.xlabel('Age')
plt.ylabel('Amount')
histogram

<b>Boxplot met column 'Overall'</b>:

In [ ]:
boxplot = plt.boxplot(fifa_source['Overall'], vert=False)
boxplot

<b>Scatter plot met column 'Overall' t.o.v. column 'Age'</b>:

In [ ]:
scatter = plt.scatter(fifa_source['Overall'], fifa_source['Age'])
plt.xlabel('Overall')
plt.ylabel('Age')
scatter

<b>Scatter plot met column 'Clean Wage' t.o.v. column 'Age'</b>:

In [ ]:
scatter = plt.scatter(fifa_source['Age'], fifa_source['Clean Wage'])
plt.xlabel('Age')
plt.ylabel('Wage')
scatter

## Externe dataset

Onze gekozen externe CSV-file dataset inladen: <br>
- fifa16.csv

De dataset bevat: <br>
1. Alle spelers uit FIFA 16.
2. Speelpositie en fysieke eigenschappen.
3. Persoonlijke spelersinformatie (bijv. nationaliteit, club, leeftijd, salaris etc.).

In [ ]:
extern_source = pd.read_csv('source/fifa16.csv', index_col=0)

De grootte van onze externe dataset:
- 7.18 MB
- Bevat 14881 rijen (dus 14880 spelers, want de eerste rij bevat alle kolommen)

In [ ]:
rows_extern_dataset = extern_source.shape[0] - 1
rows_extern_dataset

Een aantal rijen als voorbeeld voor hoe de externe dataset eruit ziet:

In [ ]:
extern_source.head(10)

<b>Verplichte onderzoeksvragen: </b>
   * *In hoeverre is de waarde van een speler te voorspellen aan de hand 
       van zijn belangrijkste kenmerken?*
   

<b>Concept-onderzoeksvragen: </b>
   - *In hoeverre is de positie van een spelers te voorspellen aan de hand van zijn kenmerken?*
   - *In hoeverre is de 'potential' van de destijds jonge talenten in fifa 16 (max 21 jaar oud met een goed potentie) uitgekomen in fifa 19?*

<b>Bonus (mocht er tijd over zijn)</b>: In hoeverre is het salaris van een speler te voorspellen aan de hand van zijn kenmerken? (Voorspelling: betere speler is hoger salaris, leeftijd speelt een rol, positie speelt een rol, nationaliteit speelt een rol)

# Uitwerking van de onderzoeksvragen
*Wij als team hebben ervoor gekozen om ieder onderzoeksvraag in een op zichzelf staand 'hoofdstuk' te behandelen, wij zijn er namelijk van overtuigd dat dit de notebook overzichtelijker maakt.*

### *1. "In hoeverre is de waarde van een speler te voorspellen aan de hand van zijn belangrijkste kenmerken?"*

Het is handig om eerst even de onderzoeksvraag te gaan splitsen:
- Wat weergeeft nou eigenlijk de <b>waarde</b> van een speler?
- Wat zijn de <b>"belangrijkste kenmerken"</b> van een speler?

Laten we hiervoor eerst even weer alle kolommen tonen van de FIFA 19 dataset:

In [ ]:
fifa_source.columns

Bij <b>waarde</b> zou je kunnen denken aan de kolommen:
- 'Clean Value'; de waarde van een speler in euro's.
- 'Clean Wage'; de salaris van de desbetreffende speler.
- 'Clean Release Clause'; een bedrag binnen het contract van een speler. Stel dat er wordt onderhandeld over een speler tussen 2 teams, en de team waar de speler zich momenteel bevind gaat <b>niet</b> akkoord, dan kan het team die interesse heeft de 'Release Clause' bedrag betalen. Als dit bedrag wordt betaald, <b>moet</b> het huidige team van die desbetreffende speler hem laten gaan. Dit bedrag zit over het algemeen <b>hoger</b> dan de werkelijke waarde van een speler.

In [ ]:
# ik gebruik de oorspronkelijke kolom met de juiste characters (bijv. 'M'), dit maakt het makkelijker om te lezen
fifa_source['Value']

In [ ]:
# ik gebruik de oorspronkelijke kolom met de juiste characters (bijv. 'M'), dit maakt het makkelijker om te lezen
fifa_source['Release Clause'] 

'Clean Release Clause' willen wij <b>niet</b> gaan gebruiken, aangezien dit niet de werkelijke waarde (in geld) van een speler weergeeft. Zoals je kunt zien is het bedrag hier namelijk een stuk hoger.

Dan blijven er dus 2 kolommen over;
- 'Clean Value'
- 'Clean Wage'

*Aangezien 'Clean Value' letterlijk een soort 'totale marktwaarde' van een speler weergeeft, gaan wij voor <b>'Clean Value'</b>.*

Bij <b>belangrijkste kenmerken</b> kan je denken aan vrijwel alle kenmerken van de speler, zoals:
- Weak Foot, Balance, ShotPower, Stamina, SprintSpeed, LM, CM
- etc...

Echter is er ook een kolom 'Overall':
- een getal (rating) van een speler. Dit geeft weer hoe goed een speler is t.o.v. de andere spelers binnen de dataset.<br>*(een soort 'gemiddelde' van de belangrijkste factoren/kenmerken van een speler)*

*Aangezien 'Overall' letterlijk een soort 'gemiddelde' is van alle kenmerken, gaan wij voor <b>'Overall'</b>.*

Er zijn spelers die als 'Clean Value' de waarde <b>0</b> hebben, deze spelers willen wij niet gaan gebruiken:

In [ ]:
value_filter = fifa_source['Clean Value'] != 0

fifa_source = fifa_source[value_filter]

Wij gaan namelijk de 'Clean Value' van de spelers iets aanpassen;
- in de scatterplot zien wij namelijk een <b>exponentiële groei</b>
- om het visualiseren wat duidelijker te maken, voeren we een <b>log</b> hierop uit, je wilt geen log op getal 0 uitvoeren
- bij het predicten van een nieuwe 'Clean Value' krijg je een log versie van dit getal terug, die gaan wij dan weer omzetten naar de behorende grootte om vervolgens zo te kunnen zien wat er gepredict is

In [ ]:
fifa_source['Clean Value Log'] = np.log(fifa_source['Clean Value'])

In [ ]:
scatter = hv.Scatter((fifa_source['Overall'], fifa_source['Clean Value Log']))
scatter.opts(title="'Overall' t.o.v. 'Value' (in euro's)", xlabel= 'Overall', ylabel='Value',
             bgcolor='lightgray', width=500, height=475)

scatter

Je ziet in deze *interactieve* Scatter Plot een duidelijk <b>positief correlatie</b> tussen de kolommen 'Value' en 'Overall'<br>(wat overigens ook wel te verwachten valt: *hoe hoger je overall, hoe meer je waard bent*).

- Hieruit concluderen wij dus een hoog voorspellingskracht.

Voor het voorspellen gaan wij dan ook gebruik maken van <b>Supervised Learning</b>;
- <b>Regression (lineare regressie)</b>, we gaan hier namelijk een *continue waarde* proberen te voorspellen

In [ ]:
linreg = LinearRegression()

Organiseren van de data: <b>features (eigenschappen)</b> en <b>target (resultaat)</b>:

In [ ]:
features = fifa_source[['Overall']]
target = fifa_source[['Clean Value Log']]

Het creëren van een <b>training- en validatieset</b>:

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target)

- 75% van de FIFA 19 dataset wordt gebruikt als <b>trainingset</b>.
- 25% van de FIFA 19 dataset wordt gebruikt als <b>testset</b>.

In [ ]:
trainingset = np.round(features_train.shape[0] / fifa_source.shape[0] * 100, decimals=0)
testset = np.round(features_test.shape[0] / fifa_source.shape[0] * 100, decimals=0)
print('Trainigset: {}% van de hele FIFA 19 dataset\nTestset: {}% van de hele FIFA 19 dataset'.format(trainingset, testset))

Het starten van 't <b>leerproces</b>:

In [ ]:
linreg.fit(features_train, target_train)

Een voorbeeld van een <b>predict</b>:

In [ ]:
messi_value_log = fifa_source['Clean Value Log'][0] # Messi is 94 Overall
predicted_value_log = linreg.predict([[94]]) ## Predicten met 94 Overall

messi_value = np.exp(messi_value_log)
predicted_value = np.exp(predicted_value_log[0][0])

print('Messi\'s value in log formaat: {}'.format("{:e}".format(messi_value_log)))
print('Predicted value in log formaat: {}\n'.format("{:e}".format(predicted_value_log[0][0])))

print('Messi\'s value: {}'.format("{:e}".format(messi_value)))
print('Predicted value: {}'.format("{:e}".format(predicted_value)))

Het model is nu getraint, wat we nu willen doen is het <b>valideren</b> van het model:
- hierbij gebruiken wij <b>Root Mean Squared Error</b> (RMSE)
- dit getal wil je zo laag mogelijk hebben t.o.v. de target, in dit geval dus <b>'Clean Value Log'</b>

In [ ]:
target_predict = linreg.predict(features_test)

In [ ]:
rmse = np.sqrt(metrics.mean_squared_error(target_test, target_predict))

rmse

Misschien kunnen we ook gebruik maken extra kolommen om onze <b>RMSE</b> lager te krijgen;
- laten we beginnen met de kolom <b>'Age'</b>

In [ ]:
###...

<b>Conclusie</b>: 
- *sss*

### *2. "In hoeverre is de positie van een spelers te voorspellen aan de hand van zijn kenmerken?"*

### *3. "In hoeverre is de 'potential' van de destijds jonge talenten in fifa 16 (max 21 jaar oud met een goed potentie) uitgekomen in fifa 19?"*